#### Aggregate Functions

In [1]:

import pandas as pd


technologies = {
    'Courses':["Spark","PySpark","Hadoop","Python","PySpark","Spark"],
    'Fee' :[20000,25000,26000,22000,24000,35000],
    'Tax':[30, 40, 35, 40, 60, 60],
    'Discount':[1000,2300,1200,2500,2000,2000],
    'Duration':[1,2,1,3,1,2]
              }
df = pd.DataFrame(technologies)

df.head(4)

,Courses,Fee,Tax,Discount,Duration
0,Spark,20000,30,1000,1
1,PySpark,25000,40,2300,2
2,Hadoop,26000,35,1200,1
3,Python,22000,40,2500,3


AGGREGATE-FUNCTIONS  DESCRIPTION

count()  Returns count for each group

size()   Returns size for each group

sum()    Returns total sum for each group

mean()   Returns mean for each group. Same as average()

average()Returns average for each group. Same as mean()

std()    Returns standard deviation for each group

var()    Return var for each group

sem()    Standard error of the mean of groups

describe() Returns different statistics

min()   Returns minimum value for each group

max()   Returns maximum value for each group

first() Returns first value for each group

last()  Returns last value for each group

nth()   Returns nth value for each group

In [4]:
df.groupby('Courses').agg({'Courses':'count'})

,Courses
Courses,
Hadoop,1
PySpark,2
Python,1
Spark,2


In [5]:
df.groupby('Courses').agg({'Courses':'size'})

,Courses
Courses,
Hadoop,1
PySpark,2
Python,1
Spark,2


In [6]:
df.groupby('Courses').agg({'Fee':'max'})

,Fee
Courses,
Hadoop,26000
PySpark,25000
Python,22000
Spark,35000


In [7]:
df.groupby('Courses').agg({'Fee':'min'})

,Fee
Courses,
Hadoop,26000
PySpark,24000
Python,22000
Spark,20000


In [10]:
df.groupby('Courses')['Fee'].nth(1)

Courses
PySpark    24000
Spark      35000
Name: Fee, dtype: int64

In [44]:
df = pd.read_csv("./DateAndValue.csv")

In [45]:
df

,date,Rolling
0,1/1/2020,2.0
1,3/1/2020,6.0
2,4/1/2020,9.0
3,5/1/2020,NaN
4,6/1/2020,10.0
5,10/1/2020,18.0
6,11/1/2020,20.0
7,12/1/2020,23.0
8,13/1/2020,24.0
9,15/1/2020,NaN


In [46]:
df['date'] = pd.to_datetime(df['date'])

In [47]:
min_date = df['date'].min()
max_date = df['date'].max()

In [48]:
df.set_index('date', inplace=True)

In [49]:
df = df.reindex(pd.date_range(min_date, max_date, freq='D'))

In [50]:
df = df.reset_index()

In [51]:
df

,index,Rolling
0,2020-01-01,2.0
1,2020-01-02,NaN
2,2020-01-03,NaN
3,2020-01-04,NaN
4,2020-01-05,NaN
...,...,...
331,2020-11-27,NaN
332,2020-11-28,NaN
333,2020-11-29,NaN
334,2020-11-30,NaN


In [52]:
df.ffill(axis=0)

,index,Rolling
0,2020-01-01,2.0
1,2020-01-02,2.0
2,2020-01-03,2.0
3,2020-01-04,2.0
4,2020-01-05,2.0
...,...,...
331,2020-11-27,20.0
332,2020-11-28,20.0
333,2020-11-29,20.0
334,2020-11-30,20.0


In [53]:
df = pd.read_csv("./Ticker.csv")

In [54]:
import random

# remove 20% of data randomly
pct_missing = 0.2
num_missing = int(pct_missing * len(df))
indexes = random.sample(range(len(df)), k=num_missing)
mask = [i in indexes for i in range(len(df))]

# mask the dataframe with some random NaNs
df["Close"] = df["Close"].mask(mask)

In [55]:
df

,Date,Close,Ticker
0,2021-08-20,88.940002,ORCL
1,2021-08-23,89.120003,ORCL
2,2021-08-24,NaN,ORCL
3,2021-08-25,88.620003,ORCL
4,2021-08-26,88.720001,ORCL
...,...,...,...
751,2022-08-15,143.179993,MAZN
752,2022-08-16,NaN,MAZN
753,2022-08-17,142.100006,MAZN
754,2022-08-18,142.300003,MAZN


In [59]:
df['Interpolate'] = df.sort_values(['Ticker', 'Date']).groupby('Ticker')['Close'].transform(lambda x: x.interpolate( method ='ffill'))

In [60]:
df

,Date,Close,Ticker,Interpolate
0,2021-08-20,88.940002,ORCL,88.940002
1,2021-08-23,89.120003,ORCL,89.120003
2,2021-08-24,NaN,ORCL,89.120003
3,2021-08-25,88.620003,ORCL,88.620003
4,2021-08-26,88.720001,ORCL,88.720001
...,...,...,...,...
751,2022-08-15,143.179993,MAZN,143.179993
752,2022-08-16,NaN,MAZN,143.179993
753,2022-08-17,142.100006,MAZN,142.100006
754,2022-08-18,142.300003,MAZN,142.300003


In [61]:
df['MovingAvg_30_Days'] = df.sort_values(['Ticker', 'Date']).groupby("Ticker")["Close"]\
                                                        .transform(lambda x: x.rolling(30, min_periods=1).mean())


In [62]:
df

,Date,Close,Ticker,Interpolate,MovingAvg_30_Days
0,2021-08-20,88.940002,ORCL,88.940002,88.940002
1,2021-08-23,89.120003,ORCL,89.120003,89.030002
2,2021-08-24,NaN,ORCL,89.120003,89.030002
3,2021-08-25,88.620003,ORCL,88.620003,88.893336
4,2021-08-26,88.720001,ORCL,88.720001,88.850002
...,...,...,...,...,...
751,2022-08-15,143.179993,MAZN,143.179993,124.822501
752,2022-08-16,NaN,MAZN,143.179993,125.314783
753,2022-08-17,142.100006,MAZN,142.100006,126.522175
754,2022-08-18,142.300003,MAZN,142.300003,127.651305


In [63]:
df['MaxPrice'] = df.groupby('Ticker')['Close'].transform('max')

In [64]:
df

,Date,Close,Ticker,Interpolate,MovingAvg_30_Days,MaxPrice
0,2021-08-20,88.940002,ORCL,88.940002,88.940002,103.650002
1,2021-08-23,89.120003,ORCL,89.120003,89.030002,103.650002
2,2021-08-24,NaN,ORCL,89.120003,89.030002,103.650002
3,2021-08-25,88.620003,ORCL,88.620003,88.893336,103.650002
4,2021-08-26,88.720001,ORCL,88.720001,88.850002,103.650002
...,...,...,...,...,...,...
751,2022-08-15,143.179993,MAZN,143.179993,124.822501,184.802994
752,2022-08-16,NaN,MAZN,143.179993,125.314783,184.802994
753,2022-08-17,142.100006,MAZN,142.100006,126.522175,184.802994
754,2022-08-18,142.300003,MAZN,142.300003,127.651305,184.802994


In [79]:
df['MovingSum_30_Days'] = df.sort_values(['Ticker', 'Date']).groupby("Ticker")["Close"]\
                                                        .transform(lambda x: x.rolling(30, min_periods=1).sum())


In [67]:
df

,Date,Close,Ticker,Interpolate,MovingAvg_30_Days,MaxPrice,MovingSum_30_Days
0,2021-08-20,88.940002,ORCL,88.940002,88.940002,103.650002,88.940002
1,2021-08-23,89.120003,ORCL,89.120003,89.030002,103.650002,178.060005
2,2021-08-24,NaN,ORCL,89.120003,89.030002,103.650002,178.060005
3,2021-08-25,88.620003,ORCL,88.620003,88.893336,103.650002,266.680008
4,2021-08-26,88.720001,ORCL,88.720001,88.850002,103.650002,355.400009
...,...,...,...,...,...,...,...
751,2022-08-15,143.179993,MAZN,143.179993,124.822501,184.802994,2995.740013
752,2022-08-16,NaN,MAZN,143.179993,125.314783,184.802994,2882.240013
753,2022-08-17,142.100006,MAZN,142.100006,126.522175,184.802994,2910.010017
754,2022-08-18,142.300003,MAZN,142.300003,127.651305,184.802994,2935.980018


In [77]:
df2 = df['Interpolate'].rolling(3).agg({'sum'})

In [78]:
df2

,sum
0,NaN
1,NaN
2,267.180008
3,266.860009
4,266.460007
...,...
751,427.369995
752,429.909989
753,428.459992
754,427.580002


In [88]:
df['Change%'] = df.sort_values(['Ticker', 'Date']).agg( {'MovingAvg_30_Days' :lambda x: (x.shift(1) / x ) * 100 })

In [90]:
df.sort_values(['Ticker', 'Date'])

,Date,Close,Ticker,Interpolate,MovingAvg_30_Days,MaxPrice,MovingSum_30_Days,Change%
504,2021-08-20,159.997498,MAZN,159.997498,159.997498,184.802994,159.997498,NaN
505,2021-08-23,163.293503,MAZN,163.293503,161.645500,184.802994,323.291001,98.980484
506,2021-08-24,165.289001,MAZN,165.289001,162.860001,184.802994,488.580002,99.254267
507,2021-08-25,164.959000,MAZN,164.959000,163.384750,184.802994,653.539002,99.678826
508,2021-08-26,165.800003,MAZN,165.800003,163.867801,184.802994,819.339005,99.705219
...,...,...,...,...,...,...,...,...
247,2022-08-15,79.110001,ORCL,79.110001,74.763750,103.650002,1794.330004,99.540776
248,2022-08-16,79.519997,ORCL,79.519997,75.077500,103.650002,1801.860003,99.582099
249,2022-08-17,79.250000,ORCL,79.250000,75.393750,103.650002,1809.449999,99.580536
250,2022-08-18,79.320000,ORCL,79.320000,75.705833,103.650002,1816.939997,99.587769


In [92]:
df['Change%_rank'] = df.sort_values(['Ticker', 'Date'])['Change%'].rank()

Rank - average when equal values

In [94]:
df.sort_values(['Ticker', 'Date'])

,Date,Close,Ticker,Interpolate,MovingAvg_30_Days,MaxPrice,MovingSum_30_Days,Change%,Change%_rank
504,2021-08-20,159.997498,MAZN,159.997498,159.997498,184.802994,159.997498,NaN,NaN
505,2021-08-23,163.293503,MAZN,163.293503,161.645500,184.802994,323.291001,98.980484,4.0
506,2021-08-24,165.289001,MAZN,165.289001,162.860001,184.802994,488.580002,99.254267,15.0
507,2021-08-25,164.959000,MAZN,164.959000,163.384750,184.802994,653.539002,99.678826,89.0
508,2021-08-26,165.800003,MAZN,165.800003,163.867801,184.802994,819.339005,99.705219,99.0
...,...,...,...,...,...,...,...,...,...
247,2022-08-15,79.110001,ORCL,79.110001,74.763750,103.650002,1794.330004,99.540776,52.0
248,2022-08-16,79.519997,ORCL,79.519997,75.077500,103.650002,1801.860003,99.582099,63.0
249,2022-08-17,79.250000,ORCL,79.250000,75.393750,103.650002,1809.449999,99.580536,61.0
250,2022-08-18,79.320000,ORCL,79.320000,75.705833,103.650002,1816.939997,99.587769,64.0


In [96]:
df['Change%_rank'] = df.sort_values(['Ticker', 'Date'])['Change%'].rank(method='dense')

In [98]:
df.sort_values(['Ticker', 'Date'])

,Date,Close,Ticker,Interpolate,MovingAvg_30_Days,MaxPrice,MovingSum_30_Days,Change%,Change%_rank
504,2021-08-20,159.997498,MAZN,159.997498,159.997498,184.802994,159.997498,NaN,NaN
505,2021-08-23,163.293503,MAZN,163.293503,161.645500,184.802994,323.291001,98.980484,4.0
506,2021-08-24,165.289001,MAZN,165.289001,162.860001,184.802994,488.580002,99.254267,15.0
507,2021-08-25,164.959000,MAZN,164.959000,163.384750,184.802994,653.539002,99.678826,89.0
508,2021-08-26,165.800003,MAZN,165.800003,163.867801,184.802994,819.339005,99.705219,99.0
...,...,...,...,...,...,...,...,...,...
247,2022-08-15,79.110001,ORCL,79.110001,74.763750,103.650002,1794.330004,99.540776,52.0
248,2022-08-16,79.519997,ORCL,79.519997,75.077500,103.650002,1801.860003,99.582099,63.0
249,2022-08-17,79.250000,ORCL,79.250000,75.393750,103.650002,1809.449999,99.580536,61.0
250,2022-08-18,79.320000,ORCL,79.320000,75.705833,103.650002,1816.939997,99.587769,64.0
